<a href="https://colab.research.google.com/github/kimyeeun48/biasofthepress_kimyeeun_201800931_hufs/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94_ipynb_201800931_%EA%B9%80%EC%98%88%EC%9D%80_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [2]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

In [1]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 6.2MB/s 
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
# konlpy를 실행하기
import konlpy

In [4]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [5]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [15]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK

rawdata_downloaded = drive.CreateFile({'id': '1y78pD8WjKrQENFEbPkiJCFdZqCHoYfCr'})
rawdata_downloaded.GetContentFile('rawdata.csv')

In [16]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.csv')

In [17]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,일자,검색어,시기,언론사,기고자,제목
0,20170508,문재인,19대대선,한국일보,전혼잎\n기자,문재인 ‘경부선 상행선’ 유세 “대통령 돼 다시 뵙겠다”
1,20170508,문재인,19대대선,세계일보,이제원,[포토] 문재인 더불어민주당 대선후보 D-1 광화문 유세
2,20170508,문재인,19대대선,아주경제,윤주혜,"문재인 광화문 유세서 딸 깜짝 등장 ""아버지는 가장 준비된 대통령 후보"""
3,20170508,문재인,19대대선,머니투데이,이재원\n기자,"문재인, '캐스팅보트' 잡기 총력 ""충청이 대통령 결정했다"""
4,20170508,문재인,19대대선,매일경제,NaN,"선거 유세 마지막날, 문재인 ""특정 지역 국민 반쪽의 대통령 아닌 모든 지역 국민 ..."
...,...,...,...,...,...,...
2850,20170310,유시민,19대대선,울산매일,노컷뉴스,"'썰전' 유시민 ""새누리당 당명 확보는 탄핵인용 대비한 것"""
2851,20170310,유시민,19대대선,아시아경제,피혜림,"'썰전' 유시민 전원책, 자유한국당 대선주자 황교안 홍준표에 ‘글쎄’"
2852,20170310,유시민,19대대선,서울경제,한해선 기자,"JTBC ‘특집 토론’ 유시민-정두언-정태옥-박주민 출연, '탄핵심판 後 대한민국,..."
2853,20170310,유시민,19대대선,헤럴드경제,김수한 기자,"‘썰전’ 심상정, 유시민“미모의 후배 낚아채” 과거 폭로"


# 형태소 분석

In [18]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [21]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[5] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [22]:
# "형태소" 변수의 저장 내용 확인
형태소

[("'", 'Punctuation'),
 ('썰전', 'Noun'),
 ("'", 'Punctuation'),
 ('심상정', 'Noun'),
 (',', 'Punctuation'),
 ('배우', 'Noun'),
 ('김고은', 'Noun'),
 ('닮은', 'Verb'),
 ('꼴', 'Noun'),
 ('?', 'Punctuation'),
 ('유시민', 'Noun'),
 ('"', 'Punctuation'),
 ('2초', 'Number'),
 ('만', 'Foreign'),
 ('"', 'Punctuation'),
 ('단호', 'Noun')]

In [23]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,',Punctuation,유시민,1
1,썰전,Noun,유시민,1
2,',Punctuation,유시민,1
3,심상정,Noun,유시민,1
4,",",Punctuation,유시민,1
5,배우,Noun,유시민,1
6,김고은,Noun,유시민,1
7,닮은,Verb,유시민,1
8,꼴,Noun,유시민,1
9,?,Punctuation,유시민,1


In [24]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0            1 type  count
 0   문재인         Noun  문재인      1
 1     ‘      Foreign  문재인      1
 2     경     Modifier  문재인      1
 3    부선         Noun  문재인      1
 4     상         Noun  문재인      1
 5    행선         Noun  문재인      1
 6     ’  Punctuation  문재인      1
 7    유세         Noun  문재인      1
 8     “      Foreign  문재인      1
 9   대통령         Noun  문재인      1
 10    돼         Verb  문재인      1
 11   다시         Noun  문재인      1
 12  뵙겠다         Verb  문재인      1
 13    ”      Foreign  문재인      1,       0            1 type  count
 0     [  Punctuation  문재인      1
 1    포토         Noun  문재인      1
 2     ]  Punctuation  문재인      1
 3   문재인         Noun  문재인      1
 4   더불어         Verb  문재인      1
 5   민주당         Noun  문재인      1
 6    대선         Noun  문재인      1
 7    후보         Noun  문재인      1
 8     D        Alpha  문재인      1
 9     -  Punctuation  문재인      1
 10    1       Number  문재인      1
 11  광화문         Noun  문재인      1
 12   유세         Noun  문재인      1,       0      

In [25]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [26]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,문재인,Noun,문재인,1
1,‘,Foreign,문재인,1
2,경,Modifier,문재인,1
3,부선,Noun,문재인,1
4,상,Noun,문재인,1
...,...,...,...,...
11,"""",Punctuation,유시민,1
12,2초,Number,유시민,1
13,만,Foreign,유시민,1
14,"""",Punctuation,유시민,1


In [27]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [28]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,문재인,Noun,문재인,1
1,‘,Foreign,문재인,1
2,경,Modifier,문재인,1
3,부선,Noun,문재인,1
4,상,Noun,문재인,1
...,...,...,...,...
11,"""",Punctuation,유시민,1
12,2초,Number,유시민,1
13,만,Foreign,유시민,1
14,"""",Punctuation,유시민,1


In [29]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [30]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류 
\n   Foreign      문재인    2
                  안철수    1
!    Punctuation  문재인    9
                  심상정    1
                  안철수    4
                        ..
沈    Foreign      홍준표    1
劉    Foreign      홍준표    1
李    Foreign      문재인    1
％    Foreign      심상정    1
                  유시민    1
Name: 카운트, Length: 7579, dtype: int64

In [31]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [32]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류      
\n  Foreign     문재인    2
                안철수    1
!   Punctuation 문재인    9
                심상정    1
                안철수    4
...                  ...
沈   Foreign     홍준표    1
劉   Foreign     홍준표    1
李   Foreign     문재인    1
％   Foreign     심상정    1
                유시민    1

[7579 rows x 1 columns]

In [33]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')